In [7]:
!pip install langchain-core langchain langchain-community langchain-openai pypdf

In [8]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Logical Routing in RAG

In logical routing, we let the LLM (Large Language Model) choose the best route from a set of predefined options. To implement this, we first create a router using a Pydantic model and define several main routes. In the QueryRouter model, we have two fields: `datasource` and `question`. The `datasource` field tells where the query should go, with the values representing our earlier created vector stores or "general" as a fallback to send the query directly to the LLM. The `question` field holds the user's query to ensure it is directed correctly. This method makes the routing process simpler and more accurate by using the LLM's abilities.

In [9]:
def generate_vectorstores(file, dir):
    loader = PyPDFLoader(file)
    documents = loader.load()

    text_splitter  = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents)

    vectorstore = Chroma.from_documents(documents=text_chunks,
                                        embedding=OpenAIEmbeddings(),
                                        persist_directory=dir)
    vectorstore.persist()
    return vectorstore


# Create a vectorstore to answer questions about the U.S.A
vectorstore_lora = generate_vectorstores("../docs/us_states_info.pdf","../docs/vectorstore_usa")

# Create a vectorstore to answer questions about Canada
vectorstore_bert = generate_vectorstores("../docs/canada_provinces_info.pdf","../docs/data/vectorstore_canada")

ValueError: File path ../docs/us_states_info.pdf is not a valid file or url

# Semantic Routing in RAG

Sometimes, we maintain multiple indexes for different domains, allowing us to query specific subsets of these indexes based on the questions we receive. For instance, imagine we have one vector store index containing detailed information about U.S. states and another dedicated to Canadian provinces and territories. When presented with a question about a particular city, we first need to determine which state or province the question pertains to and then query the relevant documents from the appropriate index. This process of determining which index or subset of indexes to query based on the query's context and content is known as query routing. Query routing ensures that we efficiently retrieve the most relevant information by classifying and directing the query to the correct data source.

Semantic routing relies on the degree of semantic similarity between the user query and the predefined router prompts to determine the most appropriate path for processing the query. This method involves analyzing the meaning and context of the user's input and comparing it to a set of potential responses or actions defined by the router. By identifying the closest match in terms of semantic content, the system can select the optimal route for handling the query. Let's explore how to implement this approach for a RAG application.



In [ ]:
from langchain_community.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

In [ ]:
us_template = """You are a very good U.S regional expert. \
               You are great at answering questions about the U.S. \
               When you don't know the answer to a question simply say I don't know.

               Here is a question:
               {query}
               """

In [ ]:
canada_template = """You are a very good Canada regional expert. \
               You are great at answering questions about Canada. \
               When you don't know the answer to a question simply say I don't know.

               Here is a question:
               {query}
               """